In [29]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import pipes_2 as pp
from sklearn.svm import SVC   ### замена класса,меняем на другое дальше



In [30]:
#ДЕЛАЕМ ФАЙЛ pipes_2 вместе с standartScaler!:  ПЕРЕДУМАЛИ

In [31]:
df = pd.read_csv('air_train.csv')
df = df.drop(['Unnamed: 0','id'], axis = 1)
df = pp.myPipe.transform(df)
X = df.drop(['satisfaction'], axis = 1)
y = df.satisfaction

t = pd.read_csv('air_test.csv')
t = t.drop(['Unnamed: 0','id'], axis = 1)
t = pp.myPipe.transform(t)
X_t = t.drop(['satisfaction'], axis = 1)
y_t = t.satisfaction

In [32]:
svm_clf = SVC()    ### замена класса

In [33]:
from sklearn.preprocessing import StandardScaler

In [34]:
sc = StandardScaler()

In [35]:
X = sc.fit_transform(X)
X_t = sc.transform(X_t)

In [11]:
paramgrid = { 'kernel':['poly','rbf','linear'], 'C':[1,0.5]}   ## замена параметров

In [12]:
GS_cv = GridSearchCV(svm_clf,param_grid = paramgrid, cv = 3)
# cv - количество разбиений

In [13]:
GS_cv.fit(X,y)

C:\Users\laptop\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\laptop\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\laptop\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 903, in fit
    super().fit(
  File "C:\Users\laptop\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 238, in fit
    raise ValueError("min_samples_split must be an integer "
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\laptop\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. 

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [5, 8, 13, 17, 20],
                         'max_features': [3, 4, 6, 8, 15],
                         'min_samples_split': [1, 2, 4, 7, 11]})

In [14]:
GS_cv.best_params_

{'max_depth': 13, 'max_features': 15, 'min_samples_split': 11}

In [15]:
best_gs_tree=GS_cv.best_estimator_

In [16]:
best_gs_tree.score(X_t,y_t)

0.9493763473975978

In [17]:
best_gs_tree.score(X,y)

0.9602036495226363

In [18]:
from sklearn.metrics import f1_score
pred = best_gs_tree.predict(X_t)
f1_score = (y_t,pred)

In [20]:
from sklearn.model_selection import cross_val_score

In [21]:
cs = cross_val_score(best_gs_tree,X,y, cv= 5, scoring = 'f1')

In [22]:
cs

array([0.93975768, 0.94165536, 0.94159727, 0.94621953, 0.94366197])

In [24]:
cs.mean()

0.9425783642998244

In [ ]:
# как ускорить процесс с CVM
# ЧТОБЫ НЕ УСНУТЬ НА КРОСС-ВАЛИДАЦИИ И ПОДБОРЕ ПАРАМЕТРОВ
#

# Ускоряем CVM, Cross-validation

# мы разбиваем наши элементы на 10 частей и получаем абсолютно нормальное распределение. и потом можно тренироваться с svn и должно быть намного быстрее, если выборка очень большая

In [7]:
from sklearn.model_selection import StratifiedKFold

In [36]:
kFold = StratifiedKFold(n_splits=10)

In [37]:
iterator = kFold.split(X, y)

In [38]:
type(iterator)

generator

In [39]:
_, subset = next(iterator)

In [ ]:
#если мы нажмем снова _, subset = next(iterator), мы получим наш следующий кусочек данных и снова можем прогонять

In [40]:
subset

array([    0,     1,     2, ..., 10417, 10419, 10421])

In [41]:
subset.shape

(10391,)

In [42]:
X.shape

(103904, 22)

In [43]:
X = X[subset]
y = y[subset]

In [44]:
y.shape

(10391,)

In [45]:
X.shape

(10391, 22)

In [46]:
paramgrid = [{ 'kernel':['rbf'], 'C':[1,0.5]},
          { 'kernel':['poly'], 'degree':[3,4]} ]   ## замена параметров


#в paramgrid можно передавать список из массивов

In [48]:
GS_cv = GridSearchCV(svm_clf,param_grid = paramgrid, cv = 3)
# cv - количество разбиений

In [49]:
GS_cv.fit(X,y)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid=[{'C': [1, 0.5], 'kernel': ['rbf']},
                         {'degree': [3, 4], 'kernel': ['poly']}])

In [50]:
GS_cv.best_params_

{'C': 1, 'kernel': 'rbf'}

In [54]:
best_gs_tree=GS_cv.best_estimator_

In [55]:
best_gs_tree.score(X_t,y_t)

0.9402140437326764

In [56]:
best_gs_tree.score(X,y)

0.9501491675488404

In [57]:
from sklearn.metrics import f1_score
pred = best_gs_tree.predict(X_t)
f1_score = (y_t,pred)

# cross validation on new X and y

In [58]:
from sklearn.model_selection import cross_val_score

In [59]:
cs = cross_val_score(best_gs_tree,X,y, cv= 5, scoring = 'f1')

In [60]:
cs

array([0.92917847, 0.9216018 , 0.92359932, 0.93213685, 0.90960135])